In [2]:
import pandas as pd
import numpy as np

# Step 1: Create a synthetic loan portfolio
np.random.seed(42)

n_loans = 1000
loan_ids = np.arange(1, n_loans + 1)

# Generate synthetic features
industries = ['Retail', 'Manufacturing', 'Technology', 'Healthcare', 'Energy']
credit_ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC']
industry = np.random.choice(industries, size=n_loans)
credit_rating = np.random.choice(credit_ratings, size=n_loans, p=[0.05, 0.1, 0.15, 0.3, 0.2, 0.15, 0.05])
loan_size = np.random.uniform(100_000, 5_000_000, size=n_loans)
interest_rate = np.random.uniform(0.03, 0.12, size=n_loans)
maturity_years = np.random.randint(1, 11, size=n_loans)
collateral_value = loan_size * np.random.uniform(0.5, 1.2, size=n_loans)

# Create the DataFrame
loan_portfolio = pd.DataFrame({
    'LoanID': loan_ids,
    'Industry': industry,
    'CreditRating': credit_rating,
    'LoanSize': loan_size,
    'InterestRate': interest_rate,
    'MaturityYears': maturity_years,
    'CollateralValue': collateral_value
})

loan_portfolio.head()



,LoanID,Industry,CreditRating,LoanSize,InterestRate,MaturityYears,CollateralValue
0,1,Healthcare,B,3.609985e+06,0.036577,7,4.109148e+06
1,2,Energy,BBB,4.532132e+05,0.086011,10,2.575409e+05
2,3,Technology,BB,4.491580e+05,0.118306,8,4.928747e+05
3,4,Energy,B,1.593315e+05,0.047110,6,1.162661e+05
4,5,Energy,BB,4.786857e+06,0.101334,10,2.573609e+06


In [4]:
# Step 2: Define Macroeconomic Scenarios
macro_scenarios = {
    'Baseline': {
        'GDP_Growth': 2.0,     # in %
        'Unemployment': 4.0,   # in %
        'Interest_Rate': 3.5   # Fed funds rate or benchmark loan rate in %
    },
    'Moderate_Stress': {
        'GDP_Growth': 0.5,
        'Unemployment': 6.0,
        'Interest_Rate': 5.0
    },
    'Severe_Stress': {
        'GDP_Growth': -2.0,
        'Unemployment': 8.0,
        'Interest_Rate': 7.0
    }
}

# Convert the dictionary into a DataFrame for easier manipulation and display
macro_df = pd.DataFrame(macro_scenarios).T  # transpose to have scenarios as rows

macro_df.head()

,GDP_Growth,Unemployment,Interest_Rate
Baseline,2.0,4.0,3.5
Moderate_Stress,0.5,6.0,5.0
Severe_Stress,-2.0,8.0,7.0


In [6]:
# Step 3: Estimate PD (Probability of Default) under each scenario

# Assign base PDs using credit ratings
rating_pd_map = {
    'AAA': 0.001,
    'AA': 0.002,
    'A': 0.005,
    'BBB': 0.015,
    'BB': 0.03,
    'B': 0.07,
    'CCC': 0.15
}
loan_portfolio['Base_PD'] = loan_portfolio['CreditRating'].map(rating_pd_map)

# Adjust PDs by macro scenario multipliers
scenario_multipliers = {
    'Baseline': 1.0,
    'Moderate_Stress': 1.5,
    'Severe_Stress': 2.5
}

# Calculate PD under each macro scenario
for scenario, multiplier in scenario_multipliers.items():
    loan_portfolio[f'PD_{scenario}'] = loan_portfolio['Base_PD'] * multiplier
    loan_portfolio[f'PD_{scenario}'] = loan_portfolio[f'PD_{scenario}'].clip(upper=1.0)  # cap at 100%

loan_portfolio.head()

,LoanID,Industry,CreditRating,LoanSize,InterestRate,MaturityYears,CollateralValue,Base_PD,PD_Baseline,PD_Moderate_Stress,PD_Severe_Stress
0,1,Healthcare,B,3.609985e+06,0.036577,7,4.109148e+06,0.070,0.070,0.1050,0.1750
1,2,Energy,BBB,4.532132e+05,0.086011,10,2.575409e+05,0.015,0.015,0.0225,0.0375
2,3,Technology,BB,4.491580e+05,0.118306,8,4.928747e+05,0.030,0.030,0.0450,0.0750
3,4,Energy,B,1.593315e+05,0.047110,6,1.162661e+05,0.070,0.070,0.1050,0.1750
4,5,Energy,BB,4.786857e+06,0.101334,10,2.573609e+06,0.030,0.030,0.0450,0.0750


In [8]:
# Step 4: Estimate LGD
industry_lgd_map = {'Retail': 0.65, 'Manufacturing': 0.55, 'Technology': 0.45, 'Healthcare': 0.50, 'Energy': 0.60}
loan_portfolio['Base_LGD'] = loan_portfolio['Industry'].map(industry_lgd_map)
loan_portfolio['CoverageRatio'] = loan_portfolio['CollateralValue'] / loan_portfolio['LoanSize']
loan_portfolio['Adj_LGD'] = loan_portfolio['Base_LGD'] * (1 / loan_portfolio['CoverageRatio'])
loan_portfolio['Adj_LGD'] = loan_portfolio['Adj_LGD'].clip(upper=1.0)

loan_portfolio.head()

,LoanID,Industry,CreditRating,LoanSize,InterestRate,MaturityYears,CollateralValue,Base_PD,PD_Baseline,PD_Moderate_Stress,PD_Severe_Stress,Base_LGD,CoverageRatio,Adj_LGD
0,1,Healthcare,B,3.609985e+06,0.036577,7,4.109148e+06,0.070,0.070,0.1050,0.1750,0.50,1.138273,0.439262
1,2,Energy,BBB,4.532132e+05,0.086011,10,2.575409e+05,0.015,0.015,0.0225,0.0375,0.60,0.568255,1.000000
2,3,Technology,BB,4.491580e+05,0.118306,8,4.928747e+05,0.030,0.030,0.0450,0.0750,0.45,1.097330,0.410086
3,4,Energy,B,1.593315e+05,0.047110,6,1.162661e+05,0.070,0.070,0.1050,0.1750,0.60,0.729712,0.822242
4,5,Energy,BB,4.786857e+06,0.101334,10,2.573609e+06,0.030,0.030,0.0450,0.0750,0.60,0.537641,1.000000


In [12]:
# Step 5: Calculate Expected Loss (EL) under each macroeconomic scenario

# EL = PD × LGD × EAD (Exposure at Default)
# Here, we'll use LoanSize as a proxy for EAD

for scenario in ['Baseline', 'Moderate_Stress', 'Severe_Stress']:
    pd_col = f'PD_{scenario}'
    loan_portfolio[f'EL_{scenario}'] = loan_portfolio[pd_col] * loan_portfolio['Adj_LGD'] * loan_portfolio['LoanSize']

# Also calculate total portfolio EL under each scenario
portfolio_el_summary = loan_portfolio[[f'EL_{s}' for s in ['Baseline', 'Moderate_Stress', 'Severe_Stress']]].sum().reset_index()
portfolio_el_summary.columns = ['Scenario', 'Total_Expected_Loss']

portfolio_el_summary['Total_Expected_Loss'] = portfolio_el_summary['Total_Expected_Loss'].apply(lambda x: f"${x:,.2f}")
portfolio_el_summary.head()

,Scenario,Total_Expected_Loss
0,EL_Baseline,"$53,668,909.79"
1,EL_Moderate_Stress,"$80,503,364.68"
2,EL_Severe_Stress,"$134,172,274.47"
